In [ ]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.model import Model

import azureml.core

ws = Workspace.from_config()
wss = Webservice.list(workspace = ws, compute_type='AKS')

In [ ]:
#Your automl model name will be different than that referenced in this example, grab your model name and update this code appropriately
models = Model.list(ws)
print(models)

In [ ]:
model = Model(ws, name="AutoML101a437ac19")
print(model.name, model.id, model.version, sep = '\t')

In [ ]:
#The writefile command has to be the first line in the following block. 
# In testing this example, the "ModelDataCollector" based predictions did not seem to work and still being investigated 
# For the moment, please use this simplified prediction interface. It is possible that the ModelDataCollector may depend on azure batch under the covers 

In [ ]:
%%writefile score_dc.py

import pickle
import pandas as pd
import time
import azureml.train.automl
import json
import numpy as np
import azureml.train.automl as AutoML

def init():
    try:
        # One-time initialization of predictive model and scaler
        from azureml.core.model import Model
        from sklearn.externals import joblib
        global model

        model_name = 'AutoML101a437ac19'
        print ("Initializing model at " + time.strftime("%H:%M:%S"))
        model_path = Model.get_model_path(model_name = model_name)
        model = joblib.load(model_path)

    except Exception as e:
        print('Exception during init: ', str(e))

def run(input_json):
    try:
        inputs = json.loads(input_json)['text']
        # Get the predictions...
        prediction = model.predict(pd.DataFrame(inputs, columns = ['review']))
        #prediction = json.dumps(prediction)
        prediction = json.dumps({"result": prediction.tolist()})
        #prediction = inputs
    except Exception as e:
        prediction = str(e)
    return prediction

In [ ]:
#Optional, skip this step if the above wworks, you can create a super simple reflection scoring file to make sure the AKS cluster is working 

In [ ]:
%%writefile score_dc.py
import json


def init():
    print("This is init")


def run(data):
    test = json.loads(data)
    print(f"received data {test}")
    return f"test is {test}"

In [ ]:
# Important, the following section uses the newer "Environment" class/api to provision the docker environment. 
# The ContainerImage.image_configuration ahd ContainerImage.create style of image creation was tested but does
# not work possibly because it depends on azure batch under the covers. 

In [ ]:
from azureml.core import Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment("mytestenv2")
#myenv.python.conda_dependencies = CondaDependencies.create(conda_packages=['numpy','scikit-learn', 'packaging','py-xgboost<=0.80'])
myenv.python.conda_dependencies = CondaDependencies.create(conda_packages=['numpy','scikit-learn', 'py-xgboost<=0.80'],
                                                           pip_packages=['azureml-defaults', 'azureml-train-automl', 'azureml-monitoring','azureml-telemetry'])

# Enable Docker
docker_config = DockerConfiguration(use_docker=True)
myenv.register(ws)

In [ ]:
from azureml.core.model import InferenceConfig 

aks_config = AksWebservice.deploy_configuration(collect_model_data=True,
                                                enable_app_insights=True)
aks_service_name ='sentiment-api-with-data-coll'
aks_target = AksCompute(ws,"k8-inf-comp")

inference_config = InferenceConfig(entry_script='score_dc.py', environment=myenv)
#aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                      name=aks_service_name,
                      models=[model],
                      inference_config=inference_config,
                      deployment_config=aks_config,
                      deployment_target=aks_target,
                      overwrite=True)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())


In [ ]:
print(service.state)
print(service.get_logs())

In [ ]:
import requests
import json

key1, Key2 = service.get_keys()

headers = {'Content-Type':'application/json',
           'Authorization': 'Bearer ' + key1}

data = {"text": ['the food was horrible',
                 'wow, this movie was truely great, I totally enjoyed it!',
                 'why the heck was my package not delivered on time?', 
                 'This movie sucks!!']}
print(key1)
print(service.scoring_uri)
print(service)

resp = requests.post(service.scoring_uri, json=data, headers=headers)
print(resp)
print("Prediction Results:", resp.json())

In [ ]:
#If all goes well you will see something like 
#http://10.0.3.98:80/api/v1/service/sentiment-api-with-data-coll/score
#AksWebservice(workspace=Workspace.create(name='ngc-test-wkspace-55', subscription_id='8c98ce0c-4c4f-4ad4-8bd9-c026f79c0889', resource_group='ngcml-dev-55'), name=sentiment-api-with-data-coll, image_id=None, compute_type=None, state=AKS, scoring_uri=Healthy, tags=http://10.0.3.98:80/api/v1/service/sentiment-api-with-data-coll/score, properties=None, created_by={'hasInferenceSchema': 'False', 'hasHttps': 'False'})
#<Response [200]>
#Prediction Results: {"result": ["negative", "positive", "negative", "negative"]}